<a href="https://colab.research.google.com/github/joshdeputi2011/TestSample/blob/main/v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy pandas scikit-learn tensorflow

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import zipfile, os

# Extract uploaded ZIP
zip_file_name = next(iter(uploaded))
extract_dir = "extracted_data"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extracted files:", os.listdir(extract_dir))


In [ ]:
import pandas as pd

# Load dataset (adjust filename if needed)
csv_path = os.path.join(extract_dir, 'time_domain_features_train.csv')
df = pd.read_csv(csv_path)

# Drop UUID
df.drop(columns=["uuid"], inplace=True)

# Create status label from heart rate
df['status'] = df['HR'].apply(lambda x: 1 if x < 60 or x > 100 else 0)

# Split features and label
X = df.drop(columns=["status"])
y = df["status"]

X.head()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define model
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2)


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")


In [ ]:
model.save("health_monitor_model.h5")


In [ ]:
import gradio as gr
import numpy as np

# Define the prediction function
def predict_status(hr, sdrr, rmssd, sdsd, nn50, pnn50, sd1, sd2, s, k, skew, kurt):
    input_data = np.array([[hr, sdrr, rmssd, sdsd, nn50, pnn50, sd1, sd2, s, k, skew, kurt]])
    input_scaled = scaler.transform(input_data)
    pred = model.predict(input_scaled)[0][0]
    return "⚠️ Alert" if pred > 0.5 else "✅ Normal"

# Launch Gradio UI
gr.Interface(
    fn=predict_status,
    inputs=[
        gr.Number(label="Heart Rate (HR)"),
        gr.Number(label="SDRR"),
        gr.Number(label="RMSSD"),
        gr.Number(label="SDSD"),
        gr.Number(label="NN50"),
        gr.Number(label="pNN50"),
        gr.Number(label="SD1"),
        gr.Number(label="SD2"),
        gr.Number(label="S"),
        gr.Number(label="K"),
        gr.Number(label="Skew"),
        gr.Number(label="Kurt")
    ],
    outputs="text",
    title="Health Monitoring & Alert System",
    description="Enter heart rate variability stats to detect health alerts"
).launch()
